<a href="https://colab.research.google.com/github/androbaza/my-kaggle-submissions/blob/master/Supermarket_purchases_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd

# for visualizations
import matplotlib.pyplot as plt

import seaborn as sns
plt.style.use('fivethirtyeight')

# for defining path
import os

# for market basket analysis
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
%cd /content/drive/MyDrive/wifi-loc/magnum

/content/drive/.shortcut-targets-by-id/1c4dnYd-QEjNPYO0UiFKbCJijNn1qcJ4B/Хакатон Магнум Правила участия и Задание


# functions

In [7]:
pip install catboost squarify

In [8]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

# reading the dataset

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/wifi-loc/magnum/train_dataset.csv')
product_d = pd.read_csv('/content/drive/MyDrive/wifi-loc/magnum/itemMasterData.csv')

reduce_mem_usage(train)
reduce_mem_usage(product_d)

train_data = pd.merge(train, product_d, on=['itemID'])

# let's check the shape of the dataset
# train_data.sample(10)

In [11]:
# reading the dataset

data3 = pd.read_csv('/content/drive/MyDrive/wifi-loc/magnum/sample_output.csv.csv')

# let's check the shape of the dataset
data.shape

(3, 11)

In [19]:
# reading the dataset

data4 = pd.read_csv('/content/drive/MyDrive/wifi-loc/magnum/test_users_intermediate.csv')

# let's check the shape of the dataset
data4.shape

(10000, 1)

In [17]:
data4.head()

,userID
0,155255
1,151159
2,14900
3,155685
4,53279


In [29]:
data2.head()

,itemID,catLevel1,catLevel2,catLevel3,catLevel4
0,3688,0,1,59,0
1,28613,-1,-1,-1,-1
2,11050,0,6,2,1
3,2140,0,13,19,2
4,21130,0,7,113,3


In [18]:
data.tail()

,userID,itemID,day,shopID,quantity
37631445,275848,1717,44,7,2.000
37631446,275848,1923,26,7,0.516
37631447,275848,2204,44,7,1.000
37631448,275848,2273,26,7,1.000
37631449,275848,8956,26,7,0.516


In [ ]:
data.sample(10)

In [20]:
data.describe()

,userID,itemID,day,shopID,quantity
count,3.763145e+07,3.763145e+07,3.763145e+07,3.763145e+07,3.763145e+07
mean,1.052141e+05,3.492669e+03,4.099642e+01,1.508745e+01,1.682284e+00
std,7.179697e+04,4.367826e+03,2.420288e+01,1.107032e+01,4.515984e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e-03
25%,4.361900e+04,4.390000e+02,2.000000e+01,7.000000e+00,1.000000e+00
50%,9.463500e+04,1.855000e+03,4.000000e+01,1.400000e+01,1.000000e+00
75%,1.592530e+05,4.927000e+03,6.200000e+01,2.200000e+01,1.972000e+00
max,2.758480e+05,3.703700e+04,8.300000e+01,4.400000e+01,1.152010e+05


In [27]:
a1 = data['shopID']